# FIREX Campaign Spatial Intersect Exercise

In this exercise we are going to be looking at data from the FIREX campaign and comparing it with wildfire data shapes from the USGS. The `geopandas` lesson page will be a helpful reference. https://rwegener2.github.io/sarp_lessons/lessons/tabular_data/3_geopandas.html

In [1]:
import pandas as pd

pd.set_option('display.max_columns', 8)

## Preprocessing the FIREX Data

1. To start let's open our dataset! The file is called `firexaq-mrg60-dc8_merge_20190722_R1_thru20190905.csv` and it is located in a `data` folder with this code.

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

:::{admonition} Hint
:class: note, dropdown

This is a `csv` file so try the `pd.read_csv()` function with the filepath as an argument. Don't forget to check for metadata at the top of the file and `skiprows` if needed.

:::

_Note: This file was originally a `.ict` file but I used the File Explorer to re-name it to a `.csv` file. Pandas can open `.ict` files just fine, but I find that Excel has a hard time with them. If you rename the same file from `.ict` to `.csv`, however, Excel will open it up._

2. This file has multiple nodata values. You can see this by running `df.describe()` and looking at the minimum values. Replace the values -999999, -888888, and -66666 with `np.nan`.

:::{admonition} Hint
:class: note, dropdown

When we use `.replace()` with 1 value we use `df.replace(-9999, np.nan)`. Here we have multiple values, so we can either run the single-value version once or use a dictionary as an argument. For an example of dictionary arguments check out the **dict-like `to_replace`** section of [the `replace` docs page](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html).

:::

3. How many rows and columns are in this dataframe?

## Filtering FIREX

4. We have a pretty big dataframe. Let's filter it down to just the following columns:
```
['Fractional_Day', ' Time_Start', ' Time_Stop', ' Day_Of_Year_YANG',
       ' Latitude_YANG', ' Longitude_YANG', ' Pressure_Altitude_YANG', ' Potential_Temp_YANG', 
       ' Sat_Vapor_Press_H2O_YANG', ' CO_LGR_ppb_PEISCHL', ' Relative_Humidity_YANG', 
       ' fRH450_RH20to85_MOORE', ' H2O_LGR_ppmv_PEISCHL', ' BC_mass_90_550_nm_SCHWARZ']
```

:::{admonition} Hint
:class: note, dropdown

Google suggestion: "pandas select multiple columns"
Suggested link: [Stack Overflow](https://stackoverflow.com/questions/11285613/selecting-multiple-columns-in-a-pandas-dataframe)

:::

5. To keep trimming it, let's filter it down to just the Fractional Day 203.

:::{admonition} Hint
:class: note, dropdown

1. Remember that there are two steps in a fitler - first you use do the comparison, which returns a series of True/False values. Second you use the boolean dataframe inside of square brackets to filter the original dataframe. See the "Filtering and groupby" section of the website for more examples.

2. Since it is the first day of the 

:::

## Plotting FIREX

6. Let's start getting a look at our data! Make a scatter plot of Fractional Day vs. Pressure Altitude.

7. Add a color argument to your graph, to color the values by the mixing ratio of water vapor, H2O_LGR_ppmv_PEISCHL. Choose an interesting colormap from [the docs page](https://matplotlib.org/stable/tutorials/colors/colormaps.html) to use.

Looks like we are seeing super moist air at the surface and really dry air above. Interesting 🤔

8. Now try making a scatter plot of Pressure Altitude vs. `' BC_mass_90_550_nm_SCHWARZ'` (no color argument). When you do this, the plot will return empty. Why is that?

## Spatially Orienting FIREX

9. In order to do spatial filtering we will have to convert our pandas dataframe to a geopandas dataframe. The code to do so is shown below, but you will have to change my instances of `firex_day1` to whatever you named your filtered dataframe.

10. What is the geometry type of this dataframe (Points, Lines, Polygons, etc.)?

11. Let's get a quick look at our data on a map. Use the `df.explore()` function to get a rough idea of the flight path.

_These maps seem to take up a lot of computer memory, so when you are done looking at them it can be a good idea to "Clear Cell Outputs" on the cell._

12. Use the `.plot()` function on the geodataframe to get a quick plot of the flight path. Use the `column` argument to color the flight path by the Saturation Vapor Pressure column and the `legend=True` argument to add a colorbar.

### Congrats!

You downloaded a file, opened it, looked around, cleaned the nodata, made some plots, filtered it, organized its spatial information, and plotted it again! And all of that on a dataset that you haven't seen before. That is pretty cool 🙂

# Bonus Challenge 

## Processing the Wildfires Shapefile

1. The `data` folder has a file at `wildfires_2019_usgs/wildfires_2019_usgs.shp` which contains all of the burn areas of fires in 2019. Open that file up using `gpd.read_file()` and give the filepath to that `.shp` file as an argument. Assign output to a variable called `wildfires`.

An example of using `gpd.read_file()` is [on the docs page](https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html).

2. Look at the new dataframe. How many rows and columns does it have? What is the type of the geometry (Point, Polygon, Multipolygon, Line, etc.)?

3. What is the CRS of this dataframe?

4. Use the `df.explore()` method to get a look at the data layer.

_These maps seem to take up a lot of computer memory, so when you are done looking at them it can be a good idea to "Clear Cell Outputs" on the cell._

5. From that shape we can see that the wildfire data is spread out between the contiguous US (CONUS), Hawai'i, and Alaska. To make our file more manegable, let's cut it down to only the fires in CONUS using and intersect function, shown below.

![Intersect](https://desktop.arcgis.com/en/arcmap/latest/tools/analysis-toolbox/GUID-93B78EC9-4024-43AC-87BF-765FAD873B00-web.gif)

In order to do that we are going to create a new shape that covers the area we want to keep and intersect the two areas. We are going to create a shape that is a box using `shapely`.

In [16]:
from shapely.geometry import box

In [17]:
conus = box(-3000000, -2000000, 2000000, 2000000)  # In units of the dataset, meters

Now that we have a shape `conus` defined, use the `.intersects()` method on the `wildfires` dataframe with `conus` as an argument. This will return a True/False boolean array indicating if each row in the dataframe intersects the `conus` shape.

5. Use the True/False dataframe from the previous step to filter the `wildfires` dataframe.  Assign this dataframe to a new variable called `wildfires_conus`.

## FIREX Spatial Filtering

7. In order to intersect the two dataframes we will need to make sure that they use the same coordinate reference system. Reproject the firex dataframe to be the same CRS as the wildfires dataframe.

Hint: We have been using crs codes that start with `epsg:`. The wildfires dataframe uses a crs code that starts with `esri:`.

8. As an input into the `.intersects()` function we are going to use we want a single geometry that represents all the areas of burn in 2019 in CONUS. 

To do this use the `.unary_union` method on the geometry column of the conus wildfires data frame and assign the output to a variable called `all_fires_shp`. The [docs page](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.unary_union.html) has an additional example of using that attribute.

9. Now let's see if the flight passed over a fire at all. Use the `.intersects()` function. Buffer to 10km away from the fire.

This image visually shows what a buffer operation does to points, lines, and polygons.

![Buffers](https://pro.arcgis.com/en/pro-app/2.8/tool-reference/analysis/GUID-267CF0D1-DB92-456F-A8FE-F819981F5467-web.png)

10. We got 16 data points. Let's add a new column to the firex geodataframe called `NearFire` that holds the boolean outputs of that intersection.

11. Finally, let's look to see if we can see a discernable difference in any of the species near and far from the fire. To do this you can use a technique called groupby that can be a conceptual leap, but is really powerful. The section of the website that talks about groupby is [here](https://rwegener2.github.io/sarp_lessons/lessons/tabular_data/2_filtering_groupby.html#groupby).

When we group the `NearFire` points together and take the mean we can see that the points near the fire had much lower water saturation vapor pressure. Pretty cool!

12. To keep all this hard work let's save the output dataframe into a new file. This is a geodataframe, so we can save it to a shapefile. Shapefiles have multiple files that make them up, so its common practice to put the shapefile in its own folder.

Docs for saving a geopandas dataframe. https://geopandas.org/en/stable/docs/user_guide/io.html#writing-spatial-data

If you wanted to save your day 1 filtered pandas (non geo) data frame you could use `df.to_csv('filepath.csv')`

### Congrats!

Nice work! You have used two spatial data operations -- buffer and intersects -- in Python AND used those to spatially filter a dataframe! That's pretty cool 🙂